In [ ]:
model_path = None
rtype = "panda"
PRECISION = "FP16"

In [ ]:
import SharedArray as sa
import numpy as np
import time
import os
import sys
import shutil
import random
import time

PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
sys.path.append(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
from pkg.controller.robot_config import RobotType
from pkg.planning.filtering.lattice_model.data_utils import *
import numpy as np
int2rtypename = {v.value:v.name for v in RobotType}
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

ARM_FOLDER = ARM10_FOLDER
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

BATCH_SIZE = 1
SERVER_PERIOD = 1e-3

In [ ]:
ROBOT_TYPE_NAME = rtype
ROBOT_MODEL_ROOT = os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
last_save = sorted([item for item in os.listdir(os.path.join(ROBOT_MODEL_ROOT, last_model)) if item.startswith("model")])[-1]
model_path_rel = os.path.join(last_model, last_save)
model_log_dir = os.path.join(ROBOT_MODEL_ROOT, model_path_rel)
model_log_dir_trt = os.path.join(ROBOT_MODEL_ROOT, model_path_rel.replace("model", "trt")+"-"+PRECISION)

# Convert and save frozen graph

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(precision_mode=PRECISION)

converter = trt.TrtGraphConverterV2(input_saved_model_dir=model_log_dir,
                                    conversion_params=conversion_params)
converter.convert()

def my_input_fn():
    grasp_img_t = tf.zeros((BATCH_SIZE,) + GRASP_SHAPE + (3,), dtype=tf.float32)
    arm_img_t = tf.zeros((BATCH_SIZE,) + ARM_SHAPE + (1,), dtype=tf.float32)
    rh_mask_t = tf.zeros((BATCH_SIZE, 54), dtype=tf.float32)
    yield (grasp_img_t, arm_img_t, rh_mask_t)

In [ ]:
converter.build(input_fn=my_input_fn)

In [ ]:
converter.save(model_log_dir_trt)

# Test frozen graph

In [ ]:
grasp_img_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.grasp_img", (BATCH_SIZE,) + GRASP_SHAPE + (3,))
arm_img_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.arm_img", (BATCH_SIZE,) + ARM_SHAPE + (1,))
rh_vals_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.rh_vals", (BATCH_SIZE, 2))
result_p = sa.create(f"shm://{ROBOT_TYPE_NAME}.result", (BATCH_SIZE, 2))
query_in = sa.create(f"shm://{ROBOT_TYPE_NAME}.query_in", (1,), dtype=np.bool)
response_out = sa.create(f"shm://{ROBOT_TYPE_NAME}.response_out", (1,), dtype=np.bool)
query_quit = sa.create(f"shm://{ROBOT_TYPE_NAME}.query_quit", (1,), dtype=np.bool)
prepared_p = sa.create(f"shm://{rtype}.prepared", (1,), dtype=np.bool)

grasp_img_p[:] = 0
arm_img_p[:] = 0
rh_vals_p[:] = 0
result_p[:] = 0
query_in[0] = False
response_out[0] = False
query_quit[0] = False
rh_mask = np.zeros((BATCH_SIZE, 54))

r_mask = div_r_gaussian(rh_vals_p[0][0])
h_mask = div_h_gaussian(rh_vals_p[0][1])
rh_mask[0] = np.concatenate([r_mask, h_mask])
prepared_p[0] = True

### Call checker once to get data example. run below cell to return response so the checker can stop waiting

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

saved_model_loaded = tf.saved_model.load(
    model_log_dir_trt, tags=[tag_constants.SERVING])
graph_func = saved_model_loaded.signatures[
    signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
frozen_func = convert_variables_to_constants_v2(graph_func)

In [ ]:
query_in[0] = False
grasp_img_t = tf.constant(grasp_img_p, dtype=tf.float32)
arm_img_t = tf.constant(arm_img_p, dtype=tf.float32)
rh_mask_t = tf.constant(rh_mask, dtype=tf.float32)
input_data = (grasp_img_t, arm_img_t, rh_mask_t)
output = frozen_func(*input_data)[0].numpy()    
for i_b in range(BATCH_SIZE):
    result_p[i_b] = output[i_b]
response_out[0] = True

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
for _ in range(100):
    while not query_in[0]:
        time.sleep(SERVER_PERIOD)
    query_in[0] = False
    print("query in")
    with gtimer.block("frozen_func"):
        grasp_img_t = tf.constant(grasp_img_p, dtype=tf.float32)
        arm_img_t = tf.constant(arm_img_p, dtype=tf.float32)
        rh_mask_t = tf.constant(rh_mask, dtype=tf.float32)
        input_data = (grasp_img_t, arm_img_t, rh_mask_t)
        output = frozen_func(*input_data)[0].numpy()    
        for i_b in range(BATCH_SIZE):
            result_p[i_b] = output[i_b]
    response_out[0] = True
print(gtimer)

In [ ]:
sa.delete(f"shm://{ROBOT_TYPE_NAME}.grasp_img")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.arm_img")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.rh_vals")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.result")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.query_in")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.response_out")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.query_quit")
sa.delete(f"shm://{ROBOT_TYPE_NAME}.prepared")